# Inverse Kinematics of a 2R Planar Manipulator using Newton's Method (without damping)

In this notebook we implement the **pure Newton method** to solve the **inverse kinematics** of a 2-degree-of-freedom planar manipulator (2R).

---

## 1. Forward Kinematics

The position of the end-effector is given by:


\begin{aligned}
x(\theta) &= L_1\cos\theta_1 + L_2\cos(\theta_1+\theta_2), \\
y(\theta) &= L_1\sin\theta_1 + L_2\sin(\theta_1+\theta_2).
\end{aligned}


where:
- $L_1, L_2$ are the link lengths,
- $\theta_1, \theta_2$ are the joint angles.

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm, solve, LinAlgError
from dataclasses import dataclass

## 2. Utility Functions

We define functions for:
- **forward_xy**: forward kinematics,
- **residual**: nonlinear system $F(\theta) = 0$,
- **jacobian**: Jacobian $\partial F/\partial \theta$,
- **reachable**: workspace reachability condition,
- **normalize_angles**: keep angles within $(- \pi, \pi]$.

In [ ]:
def forward_xy(theta, L1, L2):
    t1, t2 = map(float, theta)
    x = L1*np.cos(t1) + L2*np.cos(t1 + t2)
    y = L1*np.sin(t1) + L2*np.sin(t1 + t2)
    return np.array([x, y], dtype=float)

def residual(theta, L1, L2, xd, yd):
    return forward_xy(theta, L1, L2) - np.array([xd, yd], dtype=float)

def jacobian(theta, L1, L2):
    t1, t2 = map(float, theta)
    s1, c1 = np.sin(t1), np.cos(t1)
    s12, c12 = np.sin(t1 + t2), np.cos(t1 + t2)
    return np.array([[-L1*s1 - L2*s12, -L2*s12],
                     [ L1*c1 + L2*c12,  L2*c12]], dtype=float)

def reachable(xd, yd, L1, L2):
    r = np.hypot(xd, yd)
    return abs(L1 - L2) <= r <= (L1 + L2)

def normalize_angles(theta):
    """Normalize each angle to (-pi, pi]."""
    th = np.array(theta, dtype=float)
    return (th + np.pi) % (2*np.pi) - np.pi

## 3. Pure Newton Method

Newton's method solves:

\begin{equation*}
J(\theta_k) s_k = -F(\theta_k), \qquad \theta_{k+1} = \theta_k + s_k
\end{equation*}

where:
- $F(\theta) = (x(\theta)-x_d,\; y(\theta)-y_d)$,
- $J(\theta)$ is the Jacobian,
- $s_k$ is the Newton step.

Stopping criteria:

\begin{gather*}
\|F(\theta_k)\|_\infty \leq \mathrm{tol}_f,
\\
\|\theta_{k+1}-\theta_k\|_\infty \leq \mathrm{tol}_x
\end{gather*}


In [ ]:
@dataclass
class NewtonResult:
    theta: np.ndarray
    f_norm: float
    iters: int
    converged: bool
    history: list

def newton_ik(L1, L2, xd, yd, theta0,
              tol_f=1e-10, tol_x=1e-12, kmax=50,
              clamp_angles=True, verbose=False):
    th = np.array(theta0, dtype=float)
    hist = []

    for k in range(kmax):
        F = residual(th, L1, L2, xd, yd)
        f_inf = norm(F, ord=np.inf)

        # Newton step
        J = jacobian(th, L1, L2)
        try:
            s = solve(J, -F)
        except LinAlgError:
            s = solve(J + 1e-10*np.eye(2), -F)

        th_new = th + s
        if clamp_angles:
            th_new = normalize_angles(th_new)

        hist.append({"k": k, "theta": th.copy(), "f_inf": f_inf,
                     "step_norm": norm(s)})

        if f_inf <= tol_f:
            return NewtonResult(th, f_inf, k, True, hist)
        if norm(th_new - th, ord=np.inf) <= tol_x:
            th = th_new
            F = residual(th, L1, L2, xd, yd)
            return NewtonResult(th, norm(F, ord=np.inf), k+1, True, hist)

        th = th_new

    F = residual(th, L1, L2, xd, yd)
    return NewtonResult(th, norm(F, ord=np.inf), kmax, False, hist)

## 4. Grid Exploration

To visualize the basins of attraction, we try many initial guesses $(\theta_1, \theta_2)$ in a uniform grid within $[- \pi, \pi]\times[-\pi, \pi]$.

Classification:
- `0`: did not converge,
- `1`: converged to solution with $\theta_2 < 0$ (elbow-down),
- `2`: converged to solution with $\theta_2 \geq 0$ (elbow-up).

In [ ]:
def run_grid_analysis(L1, L2, xd, yd, n=50, kmax=40, tol_f=1e-8):
    theta1_grid = np.linspace(-np.pi, np.pi, n)
    theta2_grid = np.linspace(-np.pi, np.pi, n)

    labels = np.zeros((n, n), dtype=int)
    iters_map = np.full((n, n), np.nan)

    for i, t1 in enumerate(theta1_grid):
        for j, t2 in enumerate(theta2_grid):
            res = newton_ik(L1, L2, xd, yd, theta0=np.array([t1, t2]),
                            kmax=kmax, tol_f=tol_f)
            if res.converged and res.f_norm <= tol_f:
                labels[j, i] = 1 if res.theta[1] < 0 else 2
                iters_map[j, i] = res.iters
            else:
                labels[j, i] = 0

    return labels, iters_map, theta1_grid, theta2_grid

## 5. Visualization
We display:
- **Basins of attraction**: which initial guess leads to which solution,
- **Iteration map**: how many iterations are required for convergence.

In [ ]:
def plot_basins(labels, th1, th2, title):
    plt.figure()
    plt.imshow(labels, origin='lower',
               extent=[th1[0], th1[-1], th2[0], th2[-1]],
               aspect='auto')
    plt.xlabel(r'$\\theta_1$ (rad)')
    plt.ylabel(r'$\\theta_2$ (rad)')
    plt.title(title)
    plt.colorbar(label='0=no conv, 1=down, 2=up')
    plt.show()

def plot_iters_map(iters_map, th1, th2, title):
    plt.figure()
    plt.imshow(iters_map, origin='lower',
               extent=[th1[0], th1[-1], th2[0], th2[-1]],
               aspect='auto')
    plt.xlabel(r'$\\theta_1$ (rad)')
    plt.ylabel(r'$\\theta_2$ (rad)')
    plt.title(title + " (NaN = no conv)")
    plt.colorbar(label='# iters')
    plt.show()

## 6. Main Example
We test with several targets inside the workspace.

In [ ]:
if __name__ == "__main__":
    L1, L2 = 1.0, 0.8
    targets = [(1.10, 0.60), (0.30, 1.40), (1.50, 0.10)]

    for (xd, yd) in targets:
        print(f"\n=== Analysis for target ({xd:.2f},{yd:.2f}) ===")
        if not reachable(xd, yd, L1, L2):
            print("   -> Target not reachable.")
            continue

        labels, iters_map, th1, th2 = run_grid_analysis(L1, L2, xd, yd, n=200)

        plot_basins(labels, th1, th2, f"Basins – target ({xd:.2f},{yd:.2f})")
        plot_iters_map(iters_map, th1, th2, f"Iterations – target ({xd:.2f},{yd:.2f})")